In [1]:
import textgrad as tg
import os
import networkx as nx
import csv
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
from textgrad.tasks import load_task
from textgrad.loss import MultiFieldTokenParsedEvaluation
from textgrad import Variable
import urllib
from tqdm import tqdm
from src.utils.data_utils import *
from src.data.paths_loader import load_ai_paths, load_human_paths

In [2]:
os.environ["OPENAI_API_KEY"] = "Your API Key"

llm_engine = tg.get_engine("gpt-4o-mini")
tg.set_backward_engine(tg.get_engine("gpt-4o"), override=True)

system_prompt = tg.Variable("""You are an expert in navigating articles. Your goal is to find the path as short as possible from the start article to the target article by selecting the provided links. Each time you will be given (current article, destination) pair and the the next article that you could choose. 
                            
                            Your output should include: 
                            "Choice: " followed by the title of the article you want to navigate to. 
                            "Reason: " followed by the reason why you chose that article.
                            """,
                            requires_grad=True,
                            role_description="system prompt to guide the LLM's reasoning strategy for accurate responses")

model = tg.BlackboxLLM(llm_engine, system_prompt=system_prompt)

df = pd.read_csv('wikispeedia_paths-and-graph/links.tsv', sep='\t', comment='#', header=None, names=['origin', 'destination'])
G = nx.from_pandas_edgelist(df, source='origin', target='destination', create_using=nx.DiGraph())


In [5]:
# Try before tuning the prompt

import builtins

print("Starting the process...")
ori_dest = pd.read_csv('ori_dest.csv').iloc[:, 0].tolist()
ori_dest = ori_dest
ori_dest = [builtins.eval(pair) for pair in ori_dest]
save_dir = 'paths_before_tuning'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for pair in ori_dest:
    start_article, target_article = pair
    paths = []
    for _ in range(2):
        path = generate_path(start_article, target_article, G, model)
        paths.append(path)
    with open(f'{save_dir}/{start_article}_{target_article}.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(paths)
    # calculate the average path length
    path_lengths = [len(path) for path in paths if path[-1] != 'No path found' and path[-1] != 'Invalid choice']
    if len(path_lengths) == 0:
        avg_path_length = 'No path found'
    else:
        avg_path_length = np.mean(path_lengths)
    print(f'Average path length for {start_article} to {target_article}: {avg_path_length}')
    print(f'Invalid choices: {len([path for path in paths if path[-1] == "Invalid choice"])}, No path found: {len([path for path in paths if path[-1] == "No path found"])}')
        
    

Starting the process...
Average path length for Brain to Telephone: 5.0
Invalid choices: 0, No path found: 0
Average path length for Theatre to Zebra: 8.0
Invalid choices: 0, No path found: 0
Average path length for Pyramid to Bean: 5.0
Invalid choices: 0, No path found: 0
Average path length for Batman to Wood: 5.0
Invalid choices: 0, No path found: 0
Average path length for Bird to Great_white_shark: 9.0
Invalid choices: 0, No path found: 0
Average path length for Batman to The_Holocaust: 4.0
Invalid choices: 0, No path found: 0
neighbors: ['18th_century', '19th_century', 'Anarchism', 'Anarcho-capitalism', 'Ancient_Greece', 'Aristotle', 'Augustus', 'Baruch_Spinoza', 'Bible', 'Capitalism', 'Civil_society', 'Communism', 'Confucius', 'Constitutional_monarchy', 'Corporation', 'David_Hume', 'Democracy', 'Edmund_Burke', 'Election', 'England', 'Ethics', 'Europe', 'Family', 'Force', 'France', 'French_Revolution', 'Government', 'Italy', 'Jean-Jacques_Rousseau', 'John_Locke', 'Karl_Marx', 'Kar

In [3]:
format_string = """
LLM Prompt: {system_prompt}
Query: {query}
Prediction: {pred}
Ground Truth: {target} 
Evaluation: {eval}
"""

loss_system_prompt = tg.Variable("""he LLM needs to find the article that requires possibly fewer steps to reach the destination in a graph where nodes correspond to articles and edges represent links between articles. Your task is to evaluate and provide concise feedback on the LLM's choice of the next article to navigate to.
                                 The LLM only has access to the titles of current article and destination article, and the list of article titles to choose from (the links in current article), No any other information such as the previous choices, optimal path, correct solutions. 
                                 The LLM's output should include: "Choice" which is the article it wants to access, should be exactly in the provided list, and "Reason".
                                 You will receive the <current, destination> pair, the LLM's prediction the list of optimal solutions (first steps in all possible shortest paths).
                                 You need to provide concise feedback to correct the LLM's prediction.
                                 """,
                                 role_description="System prompt to provide feedback", 
                                 requires_grad=False)

fields = {"system_prompt": None, "query": None, "pred": None, "target": None, "eval": None}

formatted_llm_call = tg.autograd.FormattedLLMCall(engine=tg.get_engine("gpt-4o"), fields=fields, format_string=format_string, system_prompt=loss_system_prompt)
optimizer = tg.TGD([system_prompt])

def loss_fn(system_prompt, query, pred, target, eval):
    return formatted_llm_call(inputs={"system_prompt": system_prompt, "query": query, "pred": pred, "target": target, "eval": eval})

In [4]:
# Larger batch size will exceede token limit in gpt-4o
batch_size = 6
losses = []
for _ in tqdm(range(24)):
    question, answer, neighbors, optimal_choices = get_question_and_answer(G)
    question = tg.Variable(question, role_description="Navigation query", requires_grad=False)
    answer = tg.Variable(answer, role_description="Optimal solutions", requires_grad=False)

    prediction = model(question)

    choice = str(prediction).split(":")[1].split("\n")[0].strip()
    if '"' in choice:
        choice = choice[1:-1]
    print(choice)
    if choice in optimal_choices:
        eval_str = "Correct as the choice is in the optimal choices"
    else:
        print(prediction)
        eval_str = f"Incorrect as the choice is not in the optimal choices, LLM response: {choice}, optimal choices: '{', '.join(optimal_choices)}'."

    eval = tg.Variable(eval_str, role_description="Evaluation of the prediction", requires_grad=False)

    loss = loss_fn(system_prompt, question, prediction, answer, eval)
    losses.append(loss)
    
    if len(losses) == batch_size:
        optimizer.zero_grad()
        total_loss = tg.sum(losses)
        total_loss.backward()
        optimizer.step()
        losses = []
        with open("4o-improve_prompts.txt", "a") as f:
            f.write(system_prompt.value + "\n")



  0%|          | 0/24 [00:00<?, ?it/s]

Renaissance


  4%|▍         | 1/24 [00:02<01:01,  2.68s/it]

Fish
Choice: Fish  
Reason: The article "Fish" is a broader category that includes sharks, and it is likely to have links to geographical locations or other related topics that could lead to "Lusaka".


  8%|▊         | 2/24 [00:05<01:01,  2.78s/it]

...

 92%|█████████▏| 22/24 [07:43<00:29, 14.60s/it]

Halloween
Choice: Halloween  
Reason: While the initial connection to "Pony" may not seem direct, "Halloween" is a culturally significant event that often features ponies in various forms, such as pony rides or themed decorations. This choice allows for a thematic exploration that could lead to connections with children's culture, festivities, and potentially to the concept of ponies in a broader context. Other options, such as tennis-related articles, do not provide a clear thematic link to "Pony." Therefore, navigating to "Halloween" may open up more relevant pathways toward the target article.


 96%|█████████▌| 23/24 [07:48<00:11, 11.68s/it]

Alfred_the_Great
Choice: "Alfred_the_Great"  
Reason: Alfred the Great is a significant historical figure associated with the 9th century, which is directly relevant to the target article "8th_century." By selecting this link, I can potentially explore connections to the historical context of the 8th century, particularly in relation to the development of the English language and culture during that time. This choice provides a more focused pathway towards understanding the historical backdrop of the 8th century, rather than diverging into broader or less relevant topics.


100%|██████████| 24/24 [10:03<00:00, 25.17s/it]


In [5]:
print(system_prompt.__str__())

You are an expert in navigating articles with the goal of finding the shortest path from the start article to the target article by selecting the provided links. Your expertise involves understanding thematic, historical, geographical, cultural, economic, and social connections. Each time you will be given a (current article, destination) pair and the list of next articles you could choose from.

Your output should include:
- "Choice: " followed by the title of the article you want to navigate to.
- "Reason: " followed by a specific and well-reasoned justification for your choice, considering direct connections and potential pathways to the target.

Consider multiple potential paths and evaluate their merits before making a choice. Use the information provided and avoid relying on assumptions or external knowledge not present in the task. Reflect on your choices and learn from any mistakes to improve your navigation strategy. Emphasize the importance of minimizing steps and prioritize 

In [6]:
import builtins

ori_dest = pd.read_csv('ori_dest.csv').iloc[:, 0].tolist()
ori_dest = [builtins.eval(pair) for pair in ori_dest]
save_dir = 'paths_after_tuning'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for pair in tqdm(ori_dest):
    start_article, target_article = pair
    paths = []
    for _ in range(2):
        path = generate_path(start_article, target_article, G, model)
        paths.append(path)
    with open(f'{save_dir}/{start_article}_{target_article}.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(paths)
    # calculate the average path length
    path_lengths = [len(path) for path in paths if path[-1] != 'No path found' and path[-1] != 'Invalid choice']
    if len(path_lengths) == 0:
        avg_path_length = 'No path found'
    else:
        avg_path_length = np.mean(path_lengths)
    print(f'Average path length for {start_article} to {target_article}: {avg_path_length}')
    print(f'Invalid choices: {len([path for path in paths if path[-1] == "Invalid choice"])}, No path found: {len([path for path in paths if path[-1] == "No path found"])}')
        
    

  1%|          | 1/159 [00:06<17:55,  6.80s/it]

Average path length for Brain to Telephone: 5.0
Invalid choices: 0, No path found: 0


  1%|▏         | 2/159 [00:21<29:42, 11.35s/it]

Average path length for Theatre to Zebra: 7.0
Invalid choices: 0, No path found: 0


  2%|▏         | 3/159 [00:34<32:11, 12.38s/it]

...

Average path length for Australian_Open to George_W._Bush: 4.0
Invalid choices: 0, No path found: 0


 99%|█████████▉| 158/159 [24:05<00:06,  6.53s/it]

Average path length for Necktie to Lion: 3.0
Invalid choices: 0, No path found: 0


100%|██████████| 159/159 [24:32<00:00,  9.26s/it]

Average path length for Sonic_the_Hedgehog_%28character%29 to Cancer: 11.0
Invalid choices: 0, No path found: 0


In [10]:
# show the defference between before and after tuning the prompt

# load the paths
paths_before = load_ai_paths('paths_before_tuning', ori_dest)
paths_after = load_ai_paths('paths_after_tuning', ori_dest)

failure_before = 0
failure_after = 0
shorter_cnt = 0
paths_length_before = []
paths_length_after = []

for pair in ori_dest:
    start_article, target_article = pair
    paths = paths_before[pair]
    path_lengths = [len(path) for path in paths if path is not None]
    if len(path_lengths) == 0:
        avg_path_length_before = 'No path found'
    else:
        avg_path_length_before = np.mean(path_lengths)
    if avg_path_length_before == 'No path found':
        failure_before += 1

    paths = paths_after[pair]
    path_lengths = [len(path) for path in paths if path is not None]
    if len(path_lengths) == 0:
        avg_path_length_after = 'No path found'
    else:
        avg_path_length_after = np.mean(path_lengths)
    if avg_path_length_after == 'No path found':
        failure_after += 1

    if avg_path_length_before == 'No path found' or avg_path_length_after == 'No path found':
        continue
    
    if avg_path_length_before > avg_path_length_after:
        shorter_cnt += 1
    elif avg_path_length_before < avg_path_length_after:
        shorter_cnt -= 1
    
    paths_length_before.append(avg_path_length_before)
    paths_length_after.append(avg_path_length_after)

    '''
    if avg_path_length_before != avg_path_length_after:
        print(f'Average path length for {start_article} to {target_article} before tuning: {avg_path_length_before}, after tuning: {avg_path_length_after}')
    '''
    
print(f'Failure rate before tuning: {failure_before/len(ori_dest)}, after tuning: {failure_after/len(ori_dest)}')
print(f'Number of shorter paths after tuning: {shorter_cnt}')

# calculate the statistics of the path length
paths_length_before = [length for length in paths_length_before if length != 'No path found']
paths_length_after = [length for length in paths_length_after if length != 'No path found']
print(f'Average path length before tuning: {np.mean(paths_length_before)}, after tuning: {np.mean(paths_length_after)}')

# statistic test
from scipy import stats
t_stat, p_val = stats.ttest_ind(paths_length_before, paths_length_after)
print(f'T-test result: t-statistic: {t_stat}, p-value: {p_val}')

Failure rate before tuning: 0.06918238993710692, after tuning: 0.08176100628930817
Number of shorter paths after tuning: -2
Average path length before tuning: 5.631205673758865, after tuning: 5.666666666666667
T-test result: t-statistic: -0.11453704991684587, p-value: 0.9088941667078448
